In [ ]:
!pip install datasets
! pip install -U accelerate==0.15.0
! pip install -U transformers==4.28.1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments,DistilBertTokenizerFast
from transformers import AutoTokenizer,TFTrainingArguments
from transformers import OpenAIGPTForSequenceClassification, OpenAIGPTTokenizer,OpenAIGPTModel
from datasets import load_dataset
from torch.utils.data import Dataset
import torch
import tensorflow as tf

In [ ]:
df = pd.read_csv('gamedev.csv')
df.drop('id', axis=1).head()

,user_id,tags,question,answer
0,4450,"['pause', 'timescale']",How can I pause my game?,"In the Editor, you can just click the pause bu..."
1,4450,"['pause', 'timescale']",What is the best way to pause my game?,"In the Editor, you can just click the pause bu..."
2,4450,"['pause', 'timescale']","When I play a game, how do I pause it?","In the Editor, you can just click the pause bu..."
3,4450,"['pause', 'timescale']",Could you please tell me how I can pause my game?,"In the Editor, you can just click the pause bu..."
4,4450,"['pause', 'timescale']","In order to pause my game, what should I do?","In the Editor, you can just click the pause bu..."


In [ ]:
X = df['question'][:1000]
y = df['answer'][:1000]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=41)
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.2, random_state=41)

In [ ]:
X_train.shape, y_train.shape, X_eval.shape, y_eval.shape, X_test.shape, y_test.shape

((720,), (720,), (180,), (180,), (100,), (100,))

In [ ]:
y_train_id2label = {id:label for id, label in enumerate(y_train)}
y_train_label2id = {label:id for id, label in enumerate(y_train)}

y_eval_id2label = {id:label for id, label in enumerate(y_eval)}
y_eval_label2id = {label:id for id, label in enumerate(y_eval)}

y_test_id2label = {id:label for id, label in enumerate(y_test)}
y_test_label2id = {label:id for id, label in enumerate(y_test)}

In [ ]:
type(X_train.to_list())

list

In [ ]:
# # def tokenize_function(examples):
# #   return tokenizer(examples["text"], padding="max_length", truncation=True)

# # tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# # train_encodings = X_train.map(tokenize_function, batched=True)
# # val_encodings = X_eval.map(tokenize_function, batched=True)
# # test_encodings = X_test.map(tokenize_function, batched=True)

# # MODEL_NAME = 'distilbert-base-uncased'
# MODEL_NAME = 'openai-gpt'
# # tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

# from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
# import torch

# tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
# model = OpenAIGPTModel.from_pretrained("openai-gpt")
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# # train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
# # val_encodings = tokenizer(list(X_eval), truncation=True, padding=True)
# # test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

# train_encodings = tokenizer(X_train.to_list(), padding="max_length",
#                                        max_length=25,
#                                        truncation=True, return_tensors="pt")
# val_encodings = tokenizer(X_eval.to_list(), padding="max_length",
#                                        max_length=25,
#                                        truncation=True, return_tensors="pt")
# test_encodings = tokenizer(X_test.to_list(), padding="max_length",
#                                        max_length=25,
#                                        truncation=True, return_tensors="pt")

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch

# Define your fine-tuning dataset class
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(self.texts[idx], truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        return {'input_ids': input_ids}

# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can also use other GPT-2 variants like "gpt2-medium", "gpt2-large", etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Define your fine-tuning dataset
train_texts = X_train.to_list()
train_dataset = CustomDataset(train_texts, tokenizer, max_length=128)

# Create DataLoader for training
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Fine-tune the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        inputs = batch['input_ids'].to(device)

        # Labels are shifted by one position
        labels = inputs[:, 1:].contiguous()

        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}')

# Save the fine-tuned model
model.save_pretrained('fine_tuned_gpt2')
tokenizer.save_pretrained('fine_tuned_gpt2')
